In [30]:
#Libraries
import pandas as pd 
import numpy as np
import json
from sklearn.preprocessing import LabelEncoder

In [31]:
encoder_classes = np.load('HateXplain/Data/classes.npy', allow_pickle=True)
encoder_two_classes = np.load('HateXplain/Data/classes_two.npy', allow_pickle=True)

encoder = LabelEncoder()
encoder.classes_ = encoder_classes

encoder_two = LabelEncoder()
encoder_two.classes_ = encoder_two_classes

print(encoder.classes_)
print(encoder_two.classes_)


['hatespeech' 'normal' 'offensive']
['non-toxic' 'toxic']


In [32]:
with open('HateXplain/Data/dataset.json', 'r') as f:
    data = json.load(f)
    
with open('HateXplain/Data/post_id_divisions.json', 'r') as f1:
    splitter = json.load(f1)


In [33]:
for i in splitter.keys():
    print(i)
    
print(len(splitter['test']))
print(len(splitter['train']))
print(len(splitter['val']))

test
train
val
1924
15383
1922


In [27]:
#for j in data.keys():
#    print(j)

post_id : Unique id for each post

🔹annotators : The list of annotations from each annotator

🔹annotators[label] : The label assigned by the annotator to this post. Possible values: [Hatespeech, Offensive, Normal]

🔹annotators[annotator_id] : The unique Id assigned to each annotator

🔹annotators[target] : A list of target community present in the post

🔹rationales : A list of rationales selected by annotators. Each rationales represents a list with values 0 or 1. A value of 1 means that the token is part of the rationale selected by the annotator. To get the particular token, we can use the same index position in "post_tokens"

🔹post_tokens : The list of tokens representing the post which was annotated

In [79]:
post_ids = []
labels = []
targets = []
part_of_sentence = []
full_texts = []


def get_part_of_sentence(rationales, post_tokens):
    #print(rationales)
    #print(post_tokens)
    parts = []
    for r in rationales:
        part = [post_tokens[k] for k, val in enumerate(r) if val == 1]
        #print(part)
        parts.append(' '.join(part))
    return parts

for split in ['train', 'val', 'test']:
    for post_id in splitter[split]:
        if post_id in data:
            post_data = data[post_id]
            full_text = ' '.join(post_data['post_tokens']) #Taking full text
            #print(full_text)
            #print(len(post_data['annotators']))
            for i, annotator in enumerate(post_data['annotators']):
                #print(annotator)

                #print(targets)
                #print(len(post_data['rationales']))
                #print(len(post_data['annotators']))
                
                if i+1 <len(post_data['rationales']):
                    part_of_sentence.append(get_part_of_sentence([post_data['rationales'][i]], post_data['post_tokens']))
                    post_ids.append(post_id)
                    labels.append(annotator['label'])
                    targets.append(', '.join(annotator['target']))
                    full_texts.append(full_text)
                else:
                    break

df = pd.DataFrame({
    'Post ID': post_ids,
    'Label': labels,
    'Target': targets,
    'Hate Text': part_of_sentence,
    'Complete Text': full_texts
})

# Display the DataFrame
print(df.head())


                       Post ID       Label                     Target  \
0                 23107796_gab  hatespeech               Hindu, Islam   
1                  9995600_gab   offensive  Minority, Refugee, Indian   
2                  9995600_gab   offensive                     Indian   
3  1227920812235051008_twitter   offensive                      Other   
4  1204931715778543624_twitter   offensive                      Hindu   

                               Hate Text  \
0            [raped neo nazi would rape]   
1                        [paki migrants]   
2                                 [paki]   
3  [chutiya retards stophindiimposition]   
4                          [hate hindus]   

                                       Complete Text  
0  u really think i would not have been raped by ...  
1  the uk has threatened to return radioactive wa...  
2  the uk has threatened to return radioactive wa...  
3  if english is not imposition then hindi is als...  
4  no liberal congrat

In [80]:
df

,Post ID,Label,Target,Hate Text,Complete Text
0,23107796_gab,hatespeech,"Hindu, Islam",[raped neo nazi would rape],u really think i would not have been raped by ...
1,9995600_gab,offensive,"Minority, Refugee, Indian",[paki migrants],the uk has threatened to return radioactive wa...
2,9995600_gab,offensive,Indian,[paki],the uk has threatened to return radioactive wa...
3,1227920812235051008_twitter,offensive,Other,[chutiya retards stophindiimposition],if english is not imposition then hindi is als...
4,1204931715778543624_twitter,offensive,Hindu,[hate hindus],no liberal congratulated hindu refugees post c...
...,...,...,...,...,...
15800,1231804803614830592_twitter,offensive,Other,[retarded],4 chan is retarded and so are you
15801,26732378_gab,hatespeech,Jewish,[kike],plenty of e celebs have covered israid sending...
15802,26732378_gab,hatespeech,Jewish,[kike],plenty of e celebs have covered israid sending...
15803,14030104_gab,hatespeech,Jewish,[kikey],yeah i got to about just over <number> followe...
